In [3]:
import cv2
import numpy as np

def auto_remove_noise(image_path, output_path):
    """
    画像からノイズ（テキストや記号）を自動検出し除去するプログラム
    テキストの特徴のみに基づいて判定を行い、位置による判定は行わない

    Args:
        image_path (str): 入力画像のファイルパス
        output_path (str): 出力画像の保存先パス

    Returns:
        numpy.ndarray: ノイズが除去された画像
    """
    # 画像の読み込み
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    
    # グレースケールに変換
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 3段階の二値化処理で異なる濃さのテキストに対応
    # THRESH_BINARY_INV: 背景が黒(0)、テキストが白(255)となる
    _, binary1 = cv2.threshold(gray, 245, 255, cv2.THRESH_BINARY_INV)  # 濃いテキスト用
    _, binary2 = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)  # 中間の濃さのテキスト用
    _, binary3 = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY_INV)  # 薄いテキスト用
    
    # 3つの二値化結果をビットごとのORで結合
    # これにより、どの閾値でも検出されたテキストを全て含める
    binary = cv2.bitwise_or(cv2.bitwise_or(binary1, binary2), binary3)
    
    # 連結成分のラベリング処理
    # num_labels: ラベルの総数
    # labels: 各ピクセルのラベル番号を持つ配列
    # stats: 各ラベル領域の統計情報（左上座標、幅、高さ、面積）
    # centroids: 各ラベル領域の重心座標
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary)
    
    # テキスト領域を記録するためのマスク画像を作成
    text_mask = np.zeros_like(gray)
    
    # 異なるサイズのテキスト領域に対するパラメータを定義
    text_regions = {
        'small': {  # 小さいテキスト（例：注釈、記号など）
            'min_area': 5,      # 最小面積（ピクセル）
            'max_area': 200,    # 最大面積（ピクセル）
            'min_ratio': 0.1,   # 最小縦横比（height/width）
            'max_ratio': 10     # 最大縦横比（height/width）
        },
        'medium': {  # 中サイズのテキスト（例：通常の文字）
            'min_area': 200,    # 最小面積（ピクセル）
            'max_area': 1000,   # 最大面積（ピクセル）
            'min_ratio': 0.1,   # 最小縦横比（height/width）
            'max_ratio': 10     # 最大縦横比（height/width）
        },
        'large': {  # 大きいテキスト（例：見出し、タイトルなど）
            'min_area': 1000,   # 最小面積（ピクセル）
            'max_area': 3000,   # 最大面積（ピクセル）
            'min_ratio': 0.05,  # 最小縦横比（height/width）
            'max_ratio': 20     # 最大縦横比（height/width）
        }
    }
    
    def is_text_like(w, h, area, region_type):
        """
        連結成分がテキストらしい特徴を持っているかを判定する

        Args:
            w (int): 領域の幅
            h (int): 領域の高さ
            area (int): 領域の面積
            region_type (str): テキスト領域のタイプ（'small', 'medium', 'large'）

        Returns:
            bool: テキストらしい特徴を持っている場合はTrue
        """
        params = text_regions[region_type]
        # 縦横比を計算（幅が0の場合は0とする）
        aspect_ratio = h / w if w > 0 else 0
        # 面積と縦横比が指定された範囲内かをチェック
        return (params['min_area'] < area < params['max_area'] and
                params['min_ratio'] < aspect_ratio < params['max_ratio'])
    
    # 各連結成分を処理
    for i in range(1, num_labels):  # 0はbackgroundなのでスキップ
        # 連結成分の統計情報を取得
        x = stats[i, cv2.CC_STAT_LEFT]      # 左端のx座標
        y = stats[i, cv2.CC_STAT_TOP]       # 上端のy座標
        w = stats[i, cv2.CC_STAT_WIDTH]     # 幅
        h = stats[i, cv2.CC_STAT_HEIGHT]    # 高さ
        area = stats[i, cv2.CC_STAT_AREA]   # 面積
        
        # 連結成分が任意のテキストカテゴリの条件を満たすかチェック
        is_noise = False
        for region_type in text_regions:
            if is_text_like(w, h, area, region_type):
                is_noise = True
                break
        
        # テキストと判定された場合、マスクに追加
        if is_noise:
            text_mask[labels == i] = 255
    
    # マスクの膨張処理
    # テキスト領域を少し広げることで、文字の周辺部分も確実に消去する
    kernel = np.ones((5,5), np.uint8)
    text_mask = cv2.dilate(text_mask, kernel, iterations=2)
    
    # 出力画像の作成
    output = image.copy()
    # マスクされた領域を白で塗りつぶし
    output[text_mask == 255] = [255, 255, 255]
    
    # 処理結果を保存
    cv2.imwrite(output_path, output)
    
    return output

def main():
    # プログラムのエントリーポイント
    input_path = '../test_data/input/test_2.jpg'
    output_path = "../test_data/input/test_3.jpg"  # 出力画像の保存先パス
    
    try:
        processed_image = auto_remove_noise(input_path, output_path)
        print(f"Successfully processed image and saved to {output_path}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Successfully processed image and saved to ../test_data/input/test_3.jpg
